In [1]:
# -----
# IMPORT LIBRAIRIES
# -----
import numpy as np
import pandas as pd
from tqdm.auto import tqdm
import plotly.express as px
from scipy.optimize import minimize, minimize_scalar
from module.batch import OptionBatch, DualDigitalBatch

# -----
# SET DISPLAY SETTINGS
# -----
pd.set_option('display.width', 50)
pd.set_option('display.max_rows', 50)
pd.set_option('display.max_columns', 50)
pd.set_option('display.float_format', lambda x: '{:,.4f}'.format(x))

In [2]:
def calculate_rv(timeseries, roll, k, q, b, r, t, kind='call', model='black_scholes'):
    
    def objective_function(x, timeseries, roll, k, q, b, r, t):
        
        option_batch = OptionBatch(extra_output_function='calculate_delta', display_progress=False)
        option_batch.build_static_rolling_batch(timeseries=timeseries, roll=roll, k=k, iv=x, q=q, b=b, r=r, t=t, kind=kind, model=model)
        
        results = option_batch.calculate_batch()
        
        delta_pnl = (results['st'].diff() * results['dst'].shift())
        option_pnl = results['pv'].diff()
        
        option_pnl.loc[option_pnl.index.get_level_values(2) == roll] = 0
        delta_pnl.loc[delta_pnl.index.get_level_values(2) == roll] = 0
        delta_pnl_sum = delta_pnl.sum()
        option_pnl_sum = option_pnl.sum()
        
        hedging_error = (delta_pnl_sum - option_pnl_sum)**2
        
        return hedging_error

    res = minimize_scalar(objective_function, bounds=(0.02, 10), args=(timeseries, roll, k, q, b, r, t))
    
    return res.x

In [3]:
def calculate_rho(timeseries, roll, k1, k2, iv1, iv2, q1, q2, b1, b2, direction1, direction2, r, t):
    
    def objective_function(x,
                           timeseries, roll, k1, k2, iv1, iv2, q1, q2, b1, b2, 
                           direction1, direction2, r, t):
        
        dual_digital_batch = DualDigitalBatch(extra_output_function='calculate_delta', display_progress=True)
        dual_digital_batch.build_static_rolling_batch(timeseries=timeseries, roll=roll, k1=k1, k2=k2, iv1=iv1, iv2=iv2, q1=q1, q2=q2, b1=b1, b2=b2, 
                                                      direction1=direction1, direction2=direction2, rho=x, r=r, t=t, notional=100, 
                                                      model='numerical_integration')
        
        results = dual_digital_batch.calculate_batch()
        
        delta1_pnl = (results['st1'].diff() * results['dst1'].shift())
        delta2_pnl = (results['st2'].diff() * results['dst2'].shift())
        delta_pnl = delta1_pnl + delta2_pnl
        option_pnl = results['pv'].diff()
        
        option_pnl.loc[option_pnl.index.get_level_values(2) == roll] = 0
        delta_pnl.loc[delta_pnl.index.get_level_values(2) == roll] = 0
        delta_pnl_sum = delta_pnl.sum()
        option_pnl_sum = option_pnl.sum()
        
        hedging_error = (delta_pnl_sum - option_pnl_sum)**2
        
        return hedging_error

    res = minimize_scalar(objective_function, bounds=(-0.99, 0.99), args=(timeseries, roll, k1, k2, iv1, iv2, q1, q2, b1, b2, 
                                                                          direction1, direction2, r, t))
    
    return res.x

In [4]:
df1 = pd.read_csv('data/SPY.csv', index_col=0, parse_dates=True).loc[:, 'Adj Close']
df2 = pd.read_csv('data/IWM.csv', index_col=0, parse_dates=True).loc[:, 'Adj Close']
df = pd.concat([df1, df2], axis=1).dropna().iloc[-251:]
df.columns = ['st1', 'st2']
iv1 = calculate_rv(timeseries=df['st1'].to_frame('st'), roll=20, k=1, q=0, b=0, r=0, t=20)
iv2 = calculate_rv(timeseries=df['st2'].to_frame('st'), roll=20, k=1, q=0, b=0, r=0, t=20)

In [5]:
df.pct_change().std() * np.sqrt(252)

st1   0.1663
st2   0.2109
dtype: float64

In [6]:
iv1, iv2

(0.15484778890962336, 0.20513878293563334)

In [7]:
calculate_rho(timeseries=df, roll=20, k1=1, k2=1, iv1=iv1, iv2=iv2, 
              q1=0, q2=0, b1=0, b2=0, direction1='up', direction2='up', r=0, t=20)

  0%|          | 0/252 [00:00<?, ?it/s]

  0%|          | 0/252 [00:00<?, ?it/s]

  0%|          | 0/252 [00:00<?, ?it/s]

  0%|          | 0/252 [00:00<?, ?it/s]

  0%|          | 0/252 [00:00<?, ?it/s]

...

  0%|          | 0/252 [00:00<?, ?it/s]

  0%|          | 0/252 [00:00<?, ?it/s]

  0%|          | 0/252 [00:00<?, ?it/s]

  0%|          | 0/252 [00:00<?, ?it/s]

0.6175661622462038

In [8]:
x

NameError: name 'x' is not defined